<a href="https://colab.research.google.com/github/ChacoGolden/Stat-Data-Science/blob/main/Panel_reg_edu_int_infl_hh_wm_age_urb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install linearmodels

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.5/94.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from linearmodels.panel import PanelOLS
from statsmodels.tools import add_constant

In [ ]:
# Load the CSV file
data = pd.read_csv('https://raw.githubusercontent.com/ChacoGolden/Stat-Data-Science/refs/heads/main/Panel_Data_Inc_Edu_Infl_Wm_Hh_Age_Urb.csv')
# Convert 'Waste' from tons to kilograms
data['Waste'] = data['Waste'] * 1000
# Inspect the first few rows of the data
print(data.head())

    Country  Year       Waste   Income  Education  Inflation  Women  \
0   Belgium  2014  484.672171  21705.0       32.6        0.5   62.9   
1  Bulgaria  2014  376.962939   3311.0       23.6       -1.6   61.5   
2   Czechia  2014  310.164578   7622.0       19.1        0.4   64.7   
3   Denmark  2014  636.289226  27861.0       29.4        0.4   71.0   
4   Germany  2014  456.714036  19733.0       23.2        0.8   72.1   

   Household   Age    Urban  
0        2.3  41.3  0.97833  
1        2.6  43.7  0.73651  
2        2.4  40.8  0.73384  
3        2.0  41.3  0.87410  
4        2.0  45.6  0.77190  


In [ ]:
# Convert 'Country' and 'Year' to categorical variables (if not already)
data['Country'] = data['Country'].astype('category')
data['Year'] = data['Year'].astype('category')

# Check for missing values
print(data.isnull().sum())
print(data.head())
num_countries = data['Country'].nunique()
print(f"Number of different countries: {num_countries}")

Country      0
Year         0
Waste        0
Income       0
Education    0
Inflation    0
Women        0
Household    0
Age          1
Urban        0
dtype: int64
    Country  Year       Waste   Income  Education  Inflation  Women  \
0   Belgium  2014  484.672171  21705.0       32.6        0.5   62.9   
1  Bulgaria  2014  376.962939   3311.0       23.6       -1.6   61.5   
2   Czechia  2014  310.164578   7622.0       19.1        0.4   64.7   
3   Denmark  2014  636.289226  27861.0       29.4        0.4   71.0   
4   Germany  2014  456.714036  19733.0       23.2        0.8   72.1   

   Household   Age    Urban  
0        2.3  41.3  0.97833  
1        2.6  43.7  0.73651  
2        2.4  40.8  0.73384  
3        2.0  41.3  0.87410  
4        2.0  45.6  0.77190  
Number of different countries: 26


Il manque les data pour l'âge moyen en Turquie

In [ ]:
data['Year'] = data['Year'].astype(int)  # Or float if years are not integers



In [ ]:
# Set 'Country' and 'Year' as index to create a MultiIndex
data = data.set_index(['Country', 'Year'])

In [ ]:
# Define the dependent variable (Y) and independent variables (X)
Y = data['Waste']
X = data[['Income', 'Education', 'Inflation', 'Women', 'Household', 'Age', 'Urban']]

# Add a constant term to the independent variables
X = add_constant(X)

In [ ]:
# Run the fixed effects model
model = PanelOLS(dependent=Y, exog=X, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

# Display the results
print(results.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                  Waste   R-squared:                        0.2628
Estimator:                   PanelOLS   R-squared (Between):             -4.8431
No. Observations:                 233   R-squared (Within):              -0.2728
Date:                Wed, Dec 04 2024   R-squared (Overall):             -3.9052
Time:                        12:38:04   Log-likelihood                   -1130.7
Cov. Estimator:                Robust                                           
                                        F-statistic:                      9.7798
Entities:                          26   P-value                           0.0000
Avg Obs:                       8.9615   Distribution:                   F(7,192)
Min Obs:                       8.0000                                           
Max Obs:                       9.0000   F-statistic (robust):             14.367
                            

/usr/local/lib/python3.10/dist-packages/linearmodels/panel/model.py:1260: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
/usr/local/lib/python3.10/dist-packages/linearmodels/panel/data.py:590: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  group_mu = self._frame.groupby(level=level).transform("mean")
/usr/local/lib/python3.10/dist-packages/linearmodels/panel/data.py:590: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  group_mu = self._frame.groupby(level=level).transform("mean")
/usr/local/l

In [ ]:
# Create the interaction term
data['Income_Education'] = data['Income'] * data['Education']

In [ ]:
# Define the dependent variable (Y) and independent variables (X)
Y = data['Waste']
X = data[['Income', 'Education', 'Inflation', 'Income_Education', 'Women', 'Household', 'Age', 'Urban']]

# Add a constant term to the independent variables
X = add_constant(X)

# Run the fixed effects model with interaction term
model = PanelOLS(dependent=Y, exog=X, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

# Display the results
print(results.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                  Waste   R-squared:                        0.2632
Estimator:                   PanelOLS   R-squared (Between):             -4.3343
No. Observations:                 233   R-squared (Within):              -0.2570
Date:                Wed, Dec 04 2024   R-squared (Overall):             -3.4936
Time:                        12:39:19   Log-likelihood                   -1130.7
Cov. Estimator:                Robust                                           
                                        F-statistic:                      8.5306
Entities:                          26   P-value                           0.0000
Avg Obs:                       8.9615   Distribution:                   F(8,191)
Min Obs:                       8.0000                                           
Max Obs:                       9.0000   F-statistic (robust):             12.671
                            

/usr/local/lib/python3.10/dist-packages/linearmodels/panel/model.py:1260: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
/usr/local/lib/python3.10/dist-packages/linearmodels/panel/data.py:590: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  group_mu = self._frame.groupby(level=level).transform("mean")
/usr/local/lib/python3.10/dist-packages/linearmodels/panel/data.py:590: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  group_mu = self._frame.groupby(level=level).transform("mean")
/usr/local/l

The interaction is negative and significant, meaning the relationship between Income and Waste weakens as Education increases.

In [ ]:
import numpy as np

# Transform 'Income' to its logarithm
data['Log_Income'] = np.log(data['Income'])


In [ ]:
# Update the interaction term to reflect the log transformation
data['Log_Income_Education'] = data['Log_Income'] * data['Education']
X['Log_Income_Education'] = data['Log_Income_Education']

# Define the dependent variable (Y) and independent variables (X)
Y = data['Waste']
X = data[['Log_Income', 'Education', 'Inflation', 'Log_Income_Education', 'Women', 'Household', 'Age', 'Urban']]



# Add a constant term to the independent variables
X = add_constant(X)

# Run the fixed effects model
model = PanelOLS(dependent=Y, exog=X, entity_effects=True, time_effects=True)
results = model.fit(cov_type='robust')

# Display the results
print(results.summary)


                          PanelOLS Estimation Summary                           
Dep. Variable:                  Waste   R-squared:                        0.2696
Estimator:                   PanelOLS   R-squared (Between):             -2.3796
No. Observations:                 233   R-squared (Within):              -0.2556
Date:                Wed, Dec 04 2024   R-squared (Overall):             -1.9271
Time:                        12:42:09   Log-likelihood                   -1129.6
Cov. Estimator:                Robust                                           
                                        F-statistic:                      8.8131
Entities:                          26   P-value                           0.0000
Avg Obs:                       8.9615   Distribution:                   F(8,191)
Min Obs:                       8.0000                                           
Max Obs:                       9.0000   F-statistic (robust):             12.959
                            

/usr/local/lib/python3.10/dist-packages/linearmodels/panel/model.py:1260: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
/usr/local/lib/python3.10/dist-packages/linearmodels/panel/data.py:590: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  group_mu = self._frame.groupby(level=level).transform("mean")
/usr/local/lib/python3.10/dist-packages/linearmodels/panel/data.py:590: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  group_mu = self._frame.groupby(level=level).transform("mean")
/usr/local/l

In [ ]:
# Log-transform 'Income' and 'Waste'
data['Log_Waste'] = np.log(data['Waste'])
data['Log_Income'] = np.log(data['Income'])
# Update the interaction term to use log-transformed income
data['Log_Income_Education'] = data['Log_Income'] * data['Education']


In [ ]:
# Define the dependent variable (Y) and independent variables (X)
Y = data['Log_Waste']
X = data[['Log_Income', 'Education', 'Inflation', 'Log_Income_Education']]

# Add a constant term
X = add_constant(X)

# Run the fixed effects model
model = PanelOLS(dependent=Y, exog=X, entity_effects=True, time_effects=True)
results = model.fit()

# Display the results
print(results.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:              Log_Waste   R-squared:                        0.1598
Estimator:                   PanelOLS   R-squared (Between):             -0.7768
No. Observations:                 234   R-squared (Within):               0.2195
Date:                Sun, Dec 01 2024   R-squared (Overall):             -0.5857
Time:                        14:05:46   Log-likelihood                    244.73
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      9.3210
Entities:                          26   P-value                           0.0000
Avg Obs:                       9.0000   Distribution:                   F(4,196)
Min Obs:                       9.0000                                           
Max Obs:                       9.0000   F-statistic (robust):             9.3210
                            

/usr/local/lib/python3.10/dist-packages/linearmodels/panel/data.py:590: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  group_mu = self._frame.groupby(level=level).transform("mean")
/usr/local/lib/python3.10/dist-packages/linearmodels/panel/data.py:590: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  group_mu = self._frame.groupby(level=level).transform("mean")
/usr/local/lib/python3.10/dist-packages/linearmodels/panel/data.py:590: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to